In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import os
import h5py
import ecg_plot
import matplotlib.pyplot as plt
from openpyxl.worksheet.datavalidation import DataValidation


In [3]:
SB = pd.read_csv('dados/SB_amostras.csv')
ST = pd.read_csv('dados/ST_amostras.csv')
AF = pd.read_csv('dados/AF_amostras.csv')
dAVB = pd.read_csv('dados/1dAVB_amostras.csv')

#dados_selecionados = pd.concat([ST, SB, AF, LBBB, RBBB, AVB], ignore_index=True)
dados_selecionados = pd.concat([ST, SB, AF, dAVB], ignore_index=True)

# SALVAR IDS EM EXCEL 
dados_selecionados = dados_selecionados.sort_values(by=['exam_id'], ascending=[True])
print ('Tamanho inicial:', dados_selecionados.shape[0])

dados_selecionados = dados_selecionados.drop_duplicates(subset='exam_id')
print ('Tamanho depois de remover duplicatas:',dados_selecionados.shape[0])

ids = dados_selecionados['exam_id'].tolist()
ids = pd.DataFrame({
    'exam_id': ids,
    'doenca': [''] * len(ids),
    'borderline': [''] * len(ids),
    'comentario': [''] * len(ids) 
})

num_parts = 5
split_dfs = np.array_split(ids, num_parts)

for i, df_part in enumerate(split_dfs):
    excel_path = f'excel/parte_{i+1}.xlsx'
    df_part.to_excel(excel_path, index=False)
    print(f"Arquivo Excel '{excel_path}' salvo com sucesso.")


Tamanho inicial: 275
Tamanho depois de remover duplicatas: 271
Arquivo Excel 'excel/amostras_1.xlsx' salvo com sucesso.
Arquivo Excel 'excel/amostras_2.xlsx' salvo com sucesso.
Arquivo Excel 'excel/amostras_3.xlsx' salvo com sucesso.
Arquivo Excel 'excel/amostras_4.xlsx' salvo com sucesso.
Arquivo Excel 'excel/amostras_5.xlsx' salvo com sucesso.


c:\Users\luisa\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
for i in range(5):
    excel_path = f'excel/parte_{i+1}.xlsx'

    # Adicionar validação de dados para a coluna 'doenca'
    wb = load_workbook(excel_path)
    ws = wb.active

    # Criar uma lista de opções para a validação de dados na coluna 'doenca'
    doenca_options = ['ST', 'SB', 'AF', 'LBBB', 'RBBB', '1dAVB', 'Outra', 'Nenhuma', 'Interferência']
    doenca_dv = DataValidation(type="list", formula1=f'"{",".join(doenca_options)}"', showDropDown=True)

    # Criar uma lista de opções para a validação de dados na coluna 'borderline'
    borderline_options = ['Sim', 'Não']
    borderline_dv = DataValidation(type="list", formula1=f'"{",".join(borderline_options)}"', showDropDown=True)

    # Adicionar a validação às colunas apropriadas
    for row in range(2, len(ids) + 2):   # Começa na linha 2 para pular o cabeçalho
        doenca_cell = ws[f'B{row}']      # Coluna B para 'doenca'
        borderline_cell = ws[f'C{row}']  # Coluna C para 'borderline'
        
        ws.add_data_validation(doenca_dv)
        ws.add_data_validation(borderline_dv)
        
        doenca_dv.add(doenca_cell)
        borderline_dv.add(borderline_cell)

    # Salvar o arquivo com a validação de dados
    wb.save(excel_path)
    print("Validação de dados adicionada e arquivo salvo com sucesso.")

Validação de dados adicionada e arquivo salvo com sucesso.
Validação de dados adicionada e arquivo salvo com sucesso.
Validação de dados adicionada e arquivo salvo com sucesso.
Validação de dados adicionada e arquivo salvo com sucesso.
Validação de dados adicionada e arquivo salvo com sucesso.


In [11]:
for i, df_part in enumerate(split_dfs):
    print(df_part.shape[0])

55
54
54
54
54


In [13]:
def create_folders(base_path, num_folders):
    for i in range(num_folders):
        folder_path = os.path.join(base_path, f'parte_{i+1}')
        os.makedirs(folder_path, exist_ok=True)

def plot_and_save_ecgs(split_dfs, arquivos_hdf5, base_path):
    create_folders(base_path, len(split_dfs))

    for i, df_part in enumerate(split_dfs):
        folder_path = os.path.join(base_path, f'parte_{i+1}')
        exam_ids_to_plot = df_part['exam_id'].tolist()
        
        for exam_id in exam_ids_to_plot:
            found = False  
            for arquivo in arquivos_hdf5:
                try:
                    with h5py.File(arquivo, 'r') as f:
                        exam_ids = np.array(f['exam_id'])
                        exam_index = np.where(exam_ids == exam_id)[0]

                        if len(exam_index) == 0:
                            continue
                        else:
                            exam_index = exam_index[0]
                            exam_tracings = f['tracings'][exam_index]
                            ecg = np.array(exam_tracings).T
                            ecg_plot.plot(ecg, sample_rate=400, title=f'ECG {exam_id}', row_height=35)
                            plt.savefig(os.path.join(folder_path, f'{exam_id}.png'), dpi=100)
                            plt.close() 
                            found = True
                            break
                except Exception as e:
                    print(f"Erro ao processar o arquivo {arquivo} para o exam ID {exam_id}: {e}")

            if not found:
                print(f"Erro: exame ID {exam_id} não encontrado em nenhum dos arquivos.")

base_path = 'imagens'

arquivos_hdf5 = [
    "../Database/filtered_exams_0_1.hdf5", "../Database/filtered_exams_2_3.hdf5",
    "../Database/filtered_exams_4_5.hdf5", "../Database/filtered_exams_6_7.hdf5",
    "../Database/filtered_exams_8_9.hdf5", "../Database/filtered_exams_10_11.hdf5",
    "../Database/filtered_exams_12_13.hdf5", "../Database/filtered_exams_14_15.hdf5",
    "../Database/filtered_exams_16_17.hdf5"
]

plot_and_save_ecgs(split_dfs, arquivos_hdf5, base_path)

print("Processo completo!")


Locator attempting to generate 1051 ticks ([-96.25, ..., 8.75]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1051 ticks ([-96.25, ..., 8.75]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1051 ticks ([-96.25, ..., 8.75]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1051 ticks ([-96.25, ..., 8.75]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1051 ticks ([-96.25, ..., 8.75]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1051 ticks ([-96.25, ..., 8.75]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1051 ticks ([-96.25, ..., 8.75]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1051 ticks ([-96.25, ..., 8.75]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1051 ticks ([-96.25, ..., 8.75]), which exceeds Locator.MAXTICKS (1000).
Locator attempting to generate 1051 ticks ([-96.25, ..., 8.75]),

Processo completo!
